In [6]:
import itertools
import random
import ast
import pandas as pd
import numpy as np
from collections import defaultdict

In [7]:
word_seq = ['ABC', 'DEF', 'GHI', 'JKL', 'ABC', 'DEF', 'JKL', 'GHI', 'ABC', 'GHI', 'DEF', 'JKL', 'ABC', 'GHI', 'JKL', 'DEF', 'ABC', 'JKL', 'DEF', 'GHI', 'ABC', 'JKL', 'GHI', 'DEF', 
 'DEF', 'ABC', 'GHI', 'JKL', 'DEF', 'ABC', 'JKL', 'GHI', 'DEF', 'GHI', 'ABC', 'JKL', 'DEF', 'GHI', 'JKL', 'ABC', 'DEF', 'JKL', 'ABC', 'GHI', 'DEF', 'JKL', 'GHI', 'ABC', 
 'GHI', 'ABC', 'DEF', 'JKL', 'GHI', 'ABC', 'JKL', 'DEF', 'GHI', 'DEF', 'ABC', 'JKL', 'GHI', 'DEF', 'JKL', 'ABC', 'GHI', 'JKL', 'ABC', 'DEF', 'GHI', 'JKL', 'DEF', 'ABC', 
 'JKL', 'ABC', 'DEF', 'GHI', 'JKL', 'ABC', 'GHI', 'DEF', 'JKL', 'DEF', 'ABC', 'GHI', 'JKL', 'DEF', 'GHI', 'ABC', 'JKL', 'GHI', 'ABC', 'DEF', 'JKL', 'GHI', 'DEF', 'ABC']

word_seq = word_seq * 2

In [10]:
def interweave(syllables, pattern):
    # Ensure syllables are broken into individual letters
    flattened_syllables = [letter for word in syllables for letter in word]
    
    # Create the interweaved sequence
    interweaved = []
    for letter, silence in zip(flattened_syllables, pattern):
        interweaved.append(letter)
        interweaved.append(silence)
    
    return interweaved

def generate_simple_pattern(length):
    units = [[1, 1, 1, 1], [1, 2, 1], [2, 1, 1], [1, 1, 2], [1, 3], [3,1], [2, 2], [4]]
    pattern = []
    while len(pattern) < length:
        pattern.extend(random.choice(units))
    print(pattern)
    return pattern[:length]

def generate_complex_pattern(length):
    base_pattern = [1, 2, 1]
    pattern = []
    increment = 0
    while len(pattern) < length:
        new_pattern = [x + increment for x in base_pattern]
        pattern.extend(new_pattern)
        increment += 1
    print(pattern)
    return pattern[:length]

def pick_pattern_template_and_syllables(condition):
    # Choose appropriate patterns based on the condition
    patterns = generate_simple_pattern(96) if condition == 'simple' else generate_complex_pattern(96)
    # Flatten the patterns if they are lists of lists
    patterns = [item for sublist in patterns for item in sublist]
    # Randomly shuffle the patterns
    random.shuffle(patterns)

    # Initialize the dictionary to keep track of metric values
    metric_values = {1: 0, 2: 0, 3: 0, 4: 0}
    selected_patterns = []
    last_metric_value = None

    for pattern in patterns:
        # Convert pattern to string if it's an integer
        pattern = str(pattern) if isinstance(pattern, int) else pattern

        # Calculate the metric value count for the current pattern
        metric_value_count = {1: pattern.count('1'), 2: pattern.count('2'), 3: pattern.count('3'), 4: pattern.count('4')}

        # Check if the first metric value in the pattern is the same as the last used metric value
        if pattern[0] == last_metric_value:
            continue

        # Check if adding the current pattern would maintain the balance of metric values
        if all(metric_values[i] + metric_value_count[i] <= len(selected_patterns) // 4 + 1 for i in range(1, 5)):
            # If the balance is maintained, add the pattern to the selected patterns and update the dictionary
            selected_patterns.append(pattern)
            for i in range(1, 5):
                metric_values[i] += metric_value_count[i]

            # Update the last used metric value
            last_metric_value = pattern[-1]

        # Stop when we have selected 576 patterns
        if len(selected_patterns) == 576:
            break

    # Combine selected patterns into one string each
    pattern = ''.join([str(x) for x in selected_patterns])
    return pattern

In [11]:
interweaved_sequence = interweave(word_seq, pick_pattern_template_and_syllables('simple'))
valid_pairs = [
    ('A', 'B'), ('B', 'C'), ('C', 'D'), ('C', 'G'), ('C', 'J'),
    ('D', 'E'), ('E', 'F'), ('F', 'G'), ('F', 'A'), ('F', 'J'),
    ('G', 'H'), ('H', 'I'), ('I', 'J'), ('I', 'A'), ('I', 'D'),
    ('J', 'K'), ('K', 'L'), ('L', 'A'), ('L', 'D'), ('L', 'G')
]
valid_pair_set = set(valid_pairs)
pair_transition_freq = defaultdict(lambda: defaultdict(int))

# Regular counting for all but the last pair
for i in range(0, len(interweaved_sequence) - 2, 2):  # Step through individual pairs
    current_pair = (interweaved_sequence[i], interweaved_sequence[i + 2])
    # Only count if current pair is valid
    if current_pair in valid_pair_set:
        silence_value = interweaved_sequence[i + 1]
        pair_transition_freq[current_pair][silence_value] += 1

# This checks if such a pair (last-first) is a valid pair to be counted
if len(interweaved_sequence) > 2:  # Ensure there are enough elements for a wrap-around
    last_pair = (interweaved_sequence[-2], interweaved_sequence[0])
    last_silence = interweaved_sequence[-1]
    if last_pair in valid_pair_set:
        pair_transition_freq[last_pair][last_silence] += 1

# Convert the frequency table to a DataFrame for analysis and display
df_pair_transitions = pd.DataFrame(pair_transition_freq).T.fillna(0).astype(int)
df_pair_transitions['Total'] = df_pair_transitions.sum(axis=1)
df_pair_transitions.sort_values('Total', ascending=False, inplace=True)
df_pair_transitions = df_pair_transitions.sort_index(axis=1)  # Sorting for better visualization and consistency

# Display the refined frequency table
print("Simple Transitional Frequencies"), print()
display(df_pair_transitions)


[2, 2, 2, 2, 1, 3, 1, 3, 2, 2, 1, 3, 4, 4, 1, 3, 1, 1, 1, 1, 2, 2, 4, 1, 1, 1, 1, 1, 3, 1, 3, 1, 3, 4, 2, 2, 1, 3, 4, 1, 1, 1, 1, 4, 1, 1, 1, 1, 4, 4, 4, 1, 1, 1, 1, 4, 4, 4, 1, 3, 1, 3, 2, 2, 4, 1, 1, 1, 1, 1, 3, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 4, 2, 2, 1, 1, 1, 1, 1, 3]


TypeError: 'int' object is not iterable

In [12]:
interweaved_sequence = interweave(word_seq, pick_pattern_template_and_syllables('complex'))
valid_pairs = [
    ('A', 'B'), ('B', 'C'), ('C', 'D'), ('C', 'G'), ('C', 'J'),
    ('D', 'E'), ('E', 'F'), ('F', 'G'), ('F', 'A'), ('F', 'J'),
    ('G', 'H'), ('H', 'I'), ('I', 'J'), ('I', 'A'), ('I', 'D'),
    ('J', 'K'), ('K', 'L'), ('L', 'A'), ('L', 'D'), ('L', 'G')
]
valid_pair_set = set(valid_pairs)
pair_transition_freq = defaultdict(lambda: defaultdict(int))

# Regular counting for all but the last pair
for i in range(0, len(interweaved_sequence) - 2, 2):  # Step through individual pairs
    current_pair = (interweaved_sequence[i], interweaved_sequence[i + 2])
    # Only count if current pair is valid
    if current_pair in valid_pair_set:
        silence_value = interweaved_sequence[i + 1]
        pair_transition_freq[current_pair][silence_value] += 1

# This checks if such a pair (last-first) is a valid pair to be counted
if len(interweaved_sequence) > 2:  # Ensure there are enough elements for a wrap-around
    last_pair = (interweaved_sequence[-2], interweaved_sequence[0])
    last_silence = interweaved_sequence[-1]
    if last_pair in valid_pair_set:
        pair_transition_freq[last_pair][last_silence] += 1

# Convert the frequency table to a DataFrame for analysis and display
df_pair_transitions = pd.DataFrame(pair_transition_freq).T.fillna(0).astype(int)
df_pair_transitions['Total'] = df_pair_transitions.sum(axis=1)
df_pair_transitions.sort_values('Total', ascending=False, inplace=True)
df_pair_transitions = df_pair_transitions.sort_index(axis=1)  # Sorting for better visualization and consistency

# Display the refined frequency table
print("Complex Transitional Frequencies"), print()
display(df_pair_transitions)


[1, 2, 1, 2, 3, 2, 3, 4, 3, 4, 5, 4, 5, 6, 5, 6, 7, 6, 7, 8, 7, 8, 9, 8, 9, 10, 9, 10, 11, 10, 11, 12, 11, 12, 13, 12, 13, 14, 13, 14, 15, 14, 15, 16, 15, 16, 17, 16, 17, 18, 17, 18, 19, 18, 19, 20, 19, 20, 21, 20, 21, 22, 21, 22, 23, 22, 23, 24, 23, 24, 25, 24, 25, 26, 25, 26, 27, 26, 27, 28, 27, 28, 29, 28, 29, 30, 29, 30, 31, 30, 31, 32, 31, 32, 33, 32]


TypeError: 'int' object is not iterable

In [35]:
def generate_pattern_file(condition, filename, num_patterns):
    # Open the file in write mode
    with open(filename, 'w') as f:
        # Generate the specified number of patterns
        for _ in range(num_patterns):
            # Generate a pattern
            pattern = pick_pattern_template_and_syllables(condition)
            # Write the pattern to the file, followed by a newline
            f.write(pattern + '\n')

# Call the function to generate a file with 1000 patterns
# generate_pattern_file('complex', 'complex_patterns.txt', 100)
# generate_pattern_file('simple', 'simple_patterns.txt', 100)

In [1]:
from collections import Counter

# Example sequence, simplified for clarity
word_seq = ['ABC', 'DEF', 'GHI', 'JKL', 'ABC', 'DEF', 'JKL', 'GHI', 'ABC', 'GHI', 'DEF', 'JKL'] * 2

# Flatten the word sequence into a continuous string of syllables
syllable_sequence = ''.join(word_seq)

# Function to count transitions
def count_transitions(seq):
    return Counter((seq[i], seq[i+1]) for i in range(len(seq) - 1))

# Counting transitions
transitions = count_transitions(syllable_sequence)
print("Transitions:", transitions)


Transitions: Counter({('A', 'B'): 6, ('B', 'C'): 6, ('D', 'E'): 6, ('E', 'F'): 6, ('G', 'H'): 6, ('H', 'I'): 6, ('J', 'K'): 6, ('K', 'L'): 6, ('C', 'D'): 4, ('F', 'J'): 4, ('L', 'A'): 3, ('F', 'G'): 2, ('I', 'J'): 2, ('L', 'G'): 2, ('I', 'A'): 2, ('C', 'G'): 2, ('I', 'D'): 2})
